# 1. About simulation of models and trajectories

The goal of this notebook is to present the basics of the simulation in our package.

Let's get familiar with the package. First, we shoud load it.

In [2]:
using MarkovProcesses

## Models

In this package, we are focused on Continuous-Time Markov Chains models that can be described by Chemical Reaction Networks.

Let's simulate our first model. We consider the famous SIR epidemiology model described by two reactions:

$$
Infection: S + I \xrightarrow{k_i} 2I \\
Recovery: I \xrightarrow{k_r} R
$$

In the MarkovProcesses package, models are objects that can be instantiatied and their types all derived from the abstract type `Model`. Let's load the SIR model.

In [2]:
load_model("SIR")

This function searchs a model file called SIR.jl in the models/ directory (at the root of the package). This files creates the necessary resources to create a `ContinuousTimeModel` and store it in a variable called SIR.

In [3]:
SIR

ContinuousTimeModel(3, 2, Dict("S" => 1,"I" => 2,"R" => 3), Dict("I" => 1), Dict("kr" => 2,"ki" => 1), Union{Nothing, String}["R1", "R2"], [0.0012, 0.05], [95, 5, 0], 0.0, MarkovProcesses.SIR_f!, ["I"], [2], MarkovProcesses.isabsorbing_SIR, Inf, 10)

This variable contains a parameter vector and an initial point. It is ready to use.

In [4]:
@show SIR.p
@show SIR.x0

SIR.p = [0.0012, 0.05]
SIR.x0 = [95, 5, 0]


3-element Array{Int64,1}:
 95
  5
  0

You can change the parameters with the function `set_param!`

In [5]:
set_param!(SIR, "ki", 0.015)
@show SIR.p
set_param!(SIR, [0.02, 0.07])
@show SIR.p

SIR.p = [0.015, 0.05]
SIR.p = [0.02, 0.07]


2-element Array{Float64,1}:
 0.02
 0.07

## Trajectories

The simulation of the model is done by the function `simulate`.

In [6]:
σ = simulate(SIR)

Trajectory(ContinuousTimeModel(3, 2, Dict("S" => 1,"I" => 2,"R" => 3), Dict("I" => 1), Dict("kr" => 2,"ki" => 1), Union{Nothing, String}["R1", "R2"], [0.02, 0.07], [95, 5, 0], 0.0, MarkovProcesses.SIR_f!, ["I"], [2], MarkovProcesses.isabsorbing_SIR, Inf, 10), [5; 6; … ; 1; 0], [0.0, 0.0002090421844976379, 0.026738149867351305, 0.1546747797181483, 0.1670337962687422, 0.1864041143902683, 0.22499455696452758, 0.2609816573473689, 0.26211457357364876, 0.34981665245812765  …  28.156954777152635, 29.8789510639123, 31.349526365163364, 31.418132567003155, 43.3115664747499, 49.84826982651157, 50.582546017297425, 51.33856085518521, 55.25637949865968, 63.081159577787396], Union{Nothing, String}[nothing, "R1", "R1", "R1", "R1", "R1", "R1", "R1", "R1", "R1"  …  "R2", "R2", "R2", "R2", "R2", "R2", "R2", "R2", "R2", "R2"])

`simulate` returns a path which type is derived from `AbstractTrajectory`. It can be either an object of type `Trajectory` for models `::ContinuousTimeModel` or `SynchronizedTrajectory` for models that includes an automaton (but this is the subject of another notebook). It is easy to access the values of a trajectory:

In [7]:
@show σ[3] # the third state of the trajectory
@show length_states(σ) # number of states
@show σ["I", 4] # Fourth value of the variable I
@show get_state_from_time(σ, 2.3)

σ[3] = [7]
length_states(σ) = 196
σ["I", 4] = 8
get_state_from_time(σ, 2.3) = [76]


1-element view(::Array{Int64,2}, 84, :) with eltype Int64:
 76

The SIR object includes an observation model symbolized by the vector `SIR.g`. Even if the variables of the model are ["S", "I", "R"], only "I" will be observed.

In [8]:
@show SIR.map_var_idx
@show SIR.g
@show size(σ.values), length(σ["I"]) # Only one column which corresponds to the I variable

SIR.map_var_idx = Dict("S" => 1,"I" => 2,"R" => 3)
SIR.g = ["I"]
(size(σ.values), length(σ["I"])) = ((196, 1), 196)


((196, 1), 196)

The SIR model is by default unbounded, i.e. each trajectory is simulated until it reaches an absorbing state.

In [9]:
@show isbounded(SIR)
@show isbounded(σ)

isbounded(SIR) = false
isbounded(σ) = false


false

For example, computations of Lp integral distances needs bounded trajectories.

In [10]:
dist_lp(simulate(SIR),simulate(SIR); p=2)

┌ Warning: Lp distance computed on unbounded trajectories. Result should be wrong
└ @ MarkovProcesses /home/moud/MarkovProcesses/markovprocesses.jl/core/trajectory.jl:61


20.942860320770663

But this feature can be changed.

In [11]:
set_time_bound!(SIR, 120.0)

120.0

In [13]:
@show dist_lp(simulate(SIR),simulate(SIR))
@show isbounded(SIR)
@show isbounded(simulate(SIR))

dist_lp(simulate(SIR), simulate(SIR)) = 270.22871331339087
isbounded(SIR) = true
isbounded(simulate(SIR)) = true


true

## Buffer size

A useful feature is that we can change the preallocation of the memory size during the simulation in order to gain computational performance.

In [13]:
load_model("ER")
@show ER.buffer_size
@timev σ = simulate(ER)
@show length_states(σ)

ER.buffer_size = 10
  0.070949 seconds (130.66 k allocations: 7.141 MiB)
elapsed time (ns): 70948721
bytes allocated:   7488324
pool allocs:       130585
non-pool GC allocs:73
length_states(σ) = 407


407

By default, the buffer size is 10. It means that a matrix of size 10 is allocated anf filled. When it is full, another matrix of size 10 is allocated. But if you know that your simulations will have a certain number of states, you can change the buffer size. It can make a big difference in terms of performance.

In [15]:
ER.buffer_size = 100
@timev σ2 = simulate(ER)
@show length_states(σ2)

  0.000232 seconds (3.02 k allocations: 212.000 KiB)
elapsed time (ns): 231641
bytes allocated:   217088
pool allocs:       3004
non-pool GC allocs:11
length_states(σ2) = 425


425

## Plots